In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Get the OpenAI API key from the .env file
load_dotenv('.env', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = openai_api_key)

In [3]:
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )
    response = completion.choices[0].message.content
    return response

In [4]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 14.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pdfplumber


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 57.2 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install pytesseract pillow


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install pdfplumber openai python-dotenv ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 141.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 162.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 175.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [23]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [24]:
import pdfplumber
from PyPDF2 import PdfReader
import ipywidgets as widgets
from IPython.display import display

In [25]:
# Step 2: File upload widget for Jupyter
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_widget)


FileUpload(value=(), accept='.pdf', description='Upload')

In [26]:
# Step 3: Save uploaded PDF file locally
def save_uploaded_file(upload_widget):
    if upload_widget.value and len(upload_widget.value) > 0:
        file_info = upload_widget.value[0]
        file_name = file_info['name']
        content = file_info['content']
        with open(file_name, "wb") as f:
            f.write(content)
        return file_name
    return None

In [27]:
# After uploading, run this cell manually
file_path = save_uploaded_file(upload_widget)
print("Saved file:", file_path)

Saved file: AFP0080 SNAP-R Print (Works as Expected).pdf


In [ ]:
# Step 4: Extract raw text from PDF using pdfplumber
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

In [29]:
# Step 5: Extract form fields (includes radio buttons / checkboxes) using PyPDF2
def extract_pdf_form_fields(file_path):
    reader = PdfReader(file_path)
    fields = reader.get_fields()
    if fields:
        result = {}
        for key, val in fields.items():
            # val can be a dictionary or direct value
            if isinstance(val, dict):
                value = val.get('/V') or val.get('/AS') or None
                # Some values are NameObjects, convert to str ignoring '/'
                if value and hasattr(value, 'original_bytes'):
                    value = value.original_bytes.decode('utf-8')
                if isinstance(value, str) and value.startswith('/'):
                    value = value[1:]
                result[key] = value
            else:
                result[key] = val
        return result
    return {}

In [30]:
# Step 6: OpenAI LLM call for JSON extraction from raw text
def get_llm_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI document parser. Output valid JSON only."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )
    return completion.choices[0].message.content

In [41]:
# Step 7: Convert PDF text to JSON using LLM with given fields
def pdf_text_to_json(raw_text):
    form_fields = extract_pdf_form_fields(file_path)
    raw_text = extract_text_from_pdf(file_path)

    # Convert form fields dictionary into a string representation for the prompt
    form_fields_str = "\n".join([f"{k}: {v}" for k,v in form_fields.items()])

    prompt = f"""
You are a document parser. Your task is to convert the provided PDF text and form data
into structured JSON.

Rules:

1. Include all fields in the output JSON, even if their value is empty or null.
Understand the file structure.
2. Input may contain checkbox fields, understand how it is represented.
For checkboxes or radio buttons:
   -Include only the items that are selected (marked with '✔' or similar).
   - If no items in a checkbox/radio group are selected, set the field to null.
   - Do NOT output boolean true/false for unselected items.
   - Extract only the items that are marked as selected, indicated by a leading '✔' or similar mark.
    - Ignore items without any selection mark.
    - If no items are selected in a checklist, set it to null.
    - Include only the items that are marked as selected, indicated by a leading '✔', or 'n' or similar mark.
  - Do NOT output boolean true/false flags for items.
  - If no items are selected in a checklist, set that checklist field to null.
  - if checkbox is under different column in table, keep it under that column name. You have follow the structure.

3. Split full names into 'FirstName' and 'LastName' if possible.
4. Group logically related fields together.
5. Output valid JSON only. No explanation, no extra text.
6.Extract data if it is split into two coloumns and/or numbered columns data

Here are the filled form fields (checkboxes, radios, etc.):

{form_fields_str}

Here is the extracted text from the PDF:

{raw_text}
"""
    return get_llm_response(prompt)

In [42]:
# Step 8: Run everything and show combined result
if file_path:
    # Extract raw text
    raw_text = extract_text_from_pdf(file_path)
    print("----- Raw Extracted Text -----")
    print(raw_text)
    
    # Extract PDF interactive form fields
    form_fields = extract_pdf_form_fields(file_path)
    print("----- PDF Form Fields -----")
    print(form_fields)

    # Convert raw text to JSON with LLM
    extracted_json = pdf_text_to_json(file_path)
    print("----- Extracted JSON from Text -----")
    print(extracted_json)
else:
    print("Please upload a PDF file above and rerun this cell.")

----- Extracted JSON from Text -----
{
  "ExportLicenseApplication": {
    "Status": "ACCEPTED",
    "ContactInformation": {
      "ReferenceNumber": "AFP0080",
      "ContactPerson": {
        "FirstName": "Zeljko",
        "LastName": "Tolo"
      },
      "TelephoneNumber": "216-317-9273",
      "FaxNumber": null,
      "Email": "ztolo@aerofp.com",
      "CreationDate": "06/19/2025",
      "TypeOfApplication": "Export License Application"
    },
    "DocumentChecklist": {
      "DocumentsSubmittedWithApplication": [
        "Export Items (BIS-748P-A)",
        "End Users (BIS-748P-B)",
        "Import/End-User Certificate",
        "Technical Specification",
        "Letter of Explanation"
      ],
      "DocumentsOnFileWithApplicant": [
        "BIS-711",
        "Letter of Assurance",
        "Nuclear Certification"
      ]
    },
    "LicenseInformation": {
      "SpecialPurpose": "Technology",
      "ResubmissionACN": null,
      "ReplacementLicenseNumber": null,
      "ImportCe